# MusicGen Fine-Tuned Models

This notebook uses fine-tuned versions of Facebook's MusicGen model.

### **Inference**

Inference for different MusicGen Fine-tuned models 

## Available Models on Hugging Face 🤗

| Model | Description | Link |
|-------|-------------|------|
| **The 1975 - artist fine tuned** | Fine-tuned for The 1975 band style | [🔗 MadJ99/musicgen-melody-the1975](https://huggingface.co/MadJ99/musicgen-melody-the1975) |
| **Indian Classical Fusion 1** | Basic Indian classical fusion model | [🔗 MadJ99/musicgen-melody-as-ch3](https://huggingface.co/MadJ99/musicgen-melody-as-ch3) |
| **Indian Classical Fusion 2** | Expanded dataset with detailed prompt | [🔗 MadJ99/musicgen-melody-as-traes](https://huggingface.co/MadJ99/musicgen-melody-as-traes) |
| **Indian Classical Fusion 3** | Metadata updated for Anoushka Shankar style | [🔗 MadJ99/musicgen-melody-traes-updated](https://huggingface.co/MadJ99/musicgen-melody-traes-updated) |
| **Indian Classical Fusion 4** | Latest model with more training data | [🔗 MadJ99/musicgen-melody-as-new-updated](https://huggingface.co/MadJ99/musicgen-melody-as-new-updated) |

## Loading Models from Hugging Face

```python
# To load models directly from Hugging Face Hub:
model, processor = load_fine_tuned_musicgen("hub:traes-updated")

# Generate music with the loaded model
prompt = "Indian classical fusion with sitar and tabla"
audio_obj, _ = generate_music(model, processor, prompt)
display(audio_obj)

In [ ]:
%pip install git+https://github.com/ylacombe/musicgen-dreamboothing demucs msclap transformers wordcloud python-dotenv

In [ ]:
import os
import re
import time

import numpy as np
import pandas as pd
import scipy.io.wavfile
import torch
from dotenv import load_dotenv
from IPython.display import Audio
from peft import PeftConfig, PeftModel
from transformers import AutoModelForTextToWaveform, AutoProcessor


In [3]:
load_dotenv()

HUGGINGFACE_USERNAME = os.getenv("HUGGINGFACE_USERNAME")
HF_TOKEN = os.getenv("HF_TOKEN")

In [4]:
def get_best_device():
    """
    Determine the best available device for PyTorch computation.
    Priority: CUDA > MPS (Apple Silicon) > CPU
    
    Returns:
        torch.device: The best available device
    """
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif hasattr(torch, 'mps') and torch.backends.mps.is_available():
        # Check for Apple Silicon GPU (M1/M2)
        return torch.device("mps")
    else:
        return torch.device("cpu")

# Set the device
device = get_best_device()
print(f"Using device: {device}")

Using device: cuda


In [5]:
def list_fine_tuned_models():
    """
    Display all available fine-tuned MusicGen models with their details.
    """
    # Define base models directory
    base_dir = "./models"

    # Model information with custom details and correct file paths
    models_info = [
        {
            "id": 1,
            "model-name": "the1975",
            "path": f"{base_dir}/the1975",
            "instance_prompt": "the 1975",
            "description": "Fine-tuned for The 1975 band style",
        },
        {
            "id": 2,
            "model-name": "as-ch3",
            "path": f"{base_dir}/as-ch3",
            "instance_prompt": "Indian classical fusion",
            "description": "Basic Indian classical fusion model",
        },
        {
            "id": 3,
            "model-name": "as-traes",
            "path": f"{base_dir}/as-traes",
            "instance_prompt": "Indian classical fusion | sitar-driven melodies | tabla rhythms | contemporary orchestral/electronic textures | raga improvisation | minimalist cross-cultural layers | mood: introspective, melancholic, meditative | dynamic tempo shifts | organic-meets-studio production.",
            "description": "Expanded dataset with detailed prompt",
        },
        {
            "id": 4,
            "model-name": "as-traes-updated",
            "path": f"{base_dir}/as-traes-updated",
            "instance_prompt": "Indian classical fusion Anoushka Shankar",
            "description": "Metadata updated for Anoushka Shankar style",
        },
        {
            "id": 5,
            "model-name": "as-new-updated",
            "path": f"{base_dir}/as-new-updated",
            "instance_prompt": "Indian classical fusion, Anoushka Shankar",
            "description": "Latest model with more training data",
        },
    ]

    # Create a DataFrame for display
    df = pd.DataFrame(models_info)

    # Create a simple table that doesn't use styling
    display_df = df[["id", "model-name", "description"]].copy()

    # Print header
    print(f"Fine-tuned MusicGen Models in {base_dir}\n")

    # Display the dataframe (no styling)
    display(display_df)

    # Print instance prompts separately for better readability
    print("\nInstance prompts for each model:")
    for i, model in enumerate(models_info):
        print(f"{model['id']}. {model['model-name']}: {model['instance_prompt']}")

    return models_info

In [8]:
# Call the function to display all models
available_models = list_fine_tuned_models()

Fine-tuned MusicGen Models in ./models



,id,model-name,description
0,1,the1975,Fine-tuned for The 1975 band style
1,2,as-ch3,Basic Indian classical fusion model
2,3,as-traes,Expanded dataset with detailed prompt
3,4,as-traes-updated,Metadata updated for Anoushka Shankar style
4,5,as-new-updated,Latest model with more training data



Instance prompts for each model:
1. the1975: the 1975
2. as-ch3: Indian classical fusion
3. as-traes: Indian classical fusion | sitar-driven melodies | tabla rhythms | contemporary orchestral/electronic textures | raga improvisation | minimalist cross-cultural layers | mood: introspective, melancholic, meditative | dynamic tempo shifts | organic-meets-studio production.
4. as-traes-updated: Indian classical fusion Anoushka Shankar
5. as-new-updated: Indian classical fusion, Anoushka Shankar


### Function to load model

In [16]:
def load_fine_tuned_musicgen(
    model_id_or_path, device="cuda", use_hub=False, organization="MadJ99", save_local=True
):
    """
    Load a fine-tuned MusicGen model with LoRA weights from local directory or Hugging Face Hub.

    Args:
        model_id_or_path: Either:
            - model ID (1-5)
            - model name (e.g., "the1975", "as-traes-updated")
            - direct path to model directory
            - "hub:" prefix followed by model name to force loading from hub
            - full hub path with organization (e.g., "SidSaxena/musicgen-melody-lora-model")
        device: Device to load the model on (default: "cuda")
        use_hub: Whether to try loading from HF Hub first (default: False)
        organization: HF organization/username for Hub models (default: "MadJ99")
        save_local: Whether to save Hub models locally after downloading (default: True)

    Returns:
        model: The loaded model with LoRA weights
        processor: The processor for the model
    """

    # Check if explicitly requesting from hub with "hub:" prefix
    if isinstance(model_id_or_path, str) and model_id_or_path.startswith("hub:"):
        use_hub = True
        model_id_or_path = model_id_or_path[4:]  # Remove the "hub:" prefix

    # Base directory for local models
    base_dir = "./models"

    # Parse organization/model_name format if provided
    if isinstance(model_id_or_path, str) and "/" in model_id_or_path:
        parts = model_id_or_path.split("/")
        if len(parts) == 2:
            organization = parts[0]
            model_name = parts[1]
            hub_model_id = model_id_or_path
        else:
            # Just a path with slashes, not org/model format
            model_name = os.path.basename(model_id_or_path)
            hub_model_id = f"{organization}/{model_name}"
    else:
        # Get model name/path
        if isinstance(model_id_or_path, int) or (
            isinstance(model_id_or_path, str) and model_id_or_path.isdigit()
        ):
            # Using model ID (1-5)
            model_id = int(model_id_or_path)
            models_info = list_fine_tuned_models()
            matching_models = [m for m in models_info if m["id"] == model_id]

            if not matching_models:
                raise ValueError(f"No model found with ID {model_id}")

            model_name = matching_models[0]["model-name"]
            model_path = os.path.join(base_dir, model_name)
            hub_model_id = f"{organization}/{model_name}"

        elif isinstance(model_id_or_path, str):
            # Check if it's a known model name
            models_info = list_fine_tuned_models()
            matching_models = [
                m for m in models_info if m["model-name"] == model_id_or_path
            ]

            if matching_models:
                # It's a known model name
                model_name = model_id_or_path
                model_path = os.path.join(base_dir, model_name)
                hub_model_id = f"{organization}/{model_name}"
            else:
                # Assume it's a direct path
                model_path = model_id_or_path
                model_name = os.path.basename(model_path)
                hub_model_id = f"{organization}/{model_name}"
        else:
            raise ValueError(f"Invalid model identifier: {model_id_or_path}")

    # Make sure model_path is defined for all paths
    if 'model_path' not in locals():
        model_path = os.path.join(base_dir, model_name)

    # Determine whether to load from hub or local
    load_from_hub = use_hub

    # If not specifically set to use hub, check if local path exists
    if not use_hub and not os.path.exists(model_path):
        print(f"Model not found locally at: {model_path}")
        print(f"Attempting to load from Hugging Face Hub...")
        load_from_hub = True

    # Load from Hugging Face Hub
    if load_from_hub:
        print(f"Loading model from Hugging Face Hub: {hub_model_id}")

        try:
            # Load configuration from the Hub
            config = PeftConfig.from_pretrained(hub_model_id)

            # Load the base model specified in the config
            model = AutoModelForTextToWaveform.from_pretrained(
                config.base_model_name_or_path, torch_dtype=torch.float16
            )

            # Load the fine-tuned LoRA weights from Hub
            model = PeftModel.from_pretrained(model, hub_model_id).to(device)

            # Load the processor from the base model
            processor = AutoProcessor.from_pretrained(config.base_model_name_or_path)

            print(f"✓ Successfully loaded model '{model_name}' from Hugging Face Hub")
            
            # Save model locally if requested
            if save_local:
                print(f"Saving model to local path: {model_path}")
                try:
                    # Create base directory if it doesn't exist
                    os.makedirs(base_dir, exist_ok=True)
                    
                    # Create model directory if it doesn't exist
                    if not os.path.exists(model_path):
                        os.makedirs(model_path, exist_ok=True)
                    
                    # Save the model files locally
                    model.save_pretrained(model_path)
                    print(f"✓ Successfully saved model to {model_path}")
                except Exception as e:
                    print(f"! Warning: Failed to save model locally: {str(e)}")
            
            return model, processor

        except Exception as e:
            if use_hub:  # Only raise if explicitly requested hub
                raise ValueError(f"Failed to load model from Hub: {str(e)}")
            print(f"Failed to load from Hub: {str(e)}")
            print(f"Falling back to local model...")

    # Load from local path
    print(f"Loading model from local path: {model_path}")

    # Check if model exists locally
    if not os.path.exists(model_path):
        raise ValueError(f"Model not found at: {model_path}")

    # Load configuration from the local path
    config = PeftConfig.from_pretrained(model_path)

    # Load the base model specified in the config
    model = AutoModelForTextToWaveform.from_pretrained(
        config.base_model_name_or_path, torch_dtype=torch.float16
    )

    # Load the fine-tuned LoRA weights and move to device
    model = PeftModel.from_pretrained(model, model_path).to(device)

    # Load the processor from the base model
    processor = AutoProcessor.from_pretrained(config.base_model_name_or_path)

    print(f"✓ Successfully loaded model '{model_name}' from local storage")
    return model, processor


# Example usage:
# 1. Load from local storage (default behavior)
# model, processor = load_fine_tuned_musicgen("as-traes-updated")

# 2. Load from Hugging Face Hub (using the hub: prefix)
# model, processor = load_fine_tuned_musicgen("hub:as-traes-updated")

# 3. Load from Hub but fall back to local if it fails
# model, processor = load_fine_tuned_musicgen("as-traes-updated", use_hub=True)

# 4. Load from Hub without saving locally
# model, processor = load_fine_tuned_musicgen("hub:as-traes-updated", save_local=False)

# 5. Load from Hub with organization specified in model path
# model, processor = load_fine_tuned_musicgen("SidSaxena/lora-indian-classical-fusion")

In [ ]:
# Load from Hugging Face Hub (using the hub: prefix)
model, processor = load_fine_tuned_musicgen("hub:traes-updated")

Fine-tuned MusicGen Models in /mnt/f/SMC/CMC/musicgen-dreamboothing/models



,id,model-name,description
0,1,the1975,Fine-tuned for The 1975 band style
1,2,as-ch3,Basic Indian classical fusion model
2,3,as-traes,Expanded dataset with detailed prompt
3,4,as-traes-updated,Metadata updated for Anoushka Shankar style
4,5,as-new-updated,Latest model with more training data



Instance prompts for each model:
1. the1975: the 1975
2. as-ch3: Indian classical fusion
3. as-traes: Indian classical fusion | sitar-driven melodies | tabla rhythms | contemporary orchestral/electronic textures | raga improvisation | minimalist cross-cultural layers | mood: introspective, melancholic, meditative | dynamic tempo shifts | organic-meets-studio production.
4. as-traes-updated: Indian classical fusion Anoushka Shankar
5. as-new-updated: Indian classical fusion, Anoushka Shankar
Loading model from Hugging Face Hub: MadJ99/musicgen-melody-traes-updated


/mnt/d/conda/envs/musicgen/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.19s/it]


✓ Successfully loaded model 'traes-updated' from Hugging Face Hub


### Function to generate music from a single prompt

In [7]:
def generate_music(
    model,
    processor,
    prompt,
    device="cuda",
    save_path=None,
    guidance_scale=3,
    max_new_tokens=1024,
    do_sample=True,
):
    # Process the text prompt
    inputs = processor(
        text=[prompt],
        padding=True,
        return_tensors="pt",
    ).to(device)

    # Print generation info
    print(f'Generating music with prompt: "{prompt}"')
    print(
        f"Parameters: guidance_scale={guidance_scale}, max_new_tokens={max_new_tokens}"
    )

    # Generate audio
    with torch.no_grad():
        audio_values = model.generate(
            **inputs,
            do_sample=do_sample,
            guidance_scale=guidance_scale,
            max_new_tokens=max_new_tokens,
        )

    # Convert to numpy array (and ensure it's float32 for saving compatibility)
    audio_np = audio_values.cpu().numpy().squeeze().astype(np.float32)

    # Create audio object for playback
    audio_obj = Audio(audio_np, rate=32000)

    # Save the audio if a path is provided
    if save_path is not None:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        # Save as WAV file
        scipy.io.wavfile.write(save_path, rate=32000, data=audio_np)
        print(f"Audio saved to: {save_path}")

    return audio_obj, audio_np

In [ ]:
# Load a model
model, processor = load_fine_tuned_musicgen("the1975")

# Generate music with the model
prompt = "slow melody music like the 1975 with guitar playing the melody with dreamy reverb and heavy drums and synth background"
audio_obj, _ = generate_music(
    model, processor, prompt, save_path="generated_music/the1975_slow_melody.wav"
)

# Display the audio
display(audio_obj)

Loading model: the1975


/mnt/d/conda/envs/musicgen/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.28s/it]


✓ Model 'the1975' loaded successfully
Generating music with prompt: "slow melody music like the 1975 with guitar playing the melody with dreamy reverb and heavy drums and synth background"
Parameters: guidance_scale=3, max_new_tokens=1024
Audio saved to: generated_music/the1975_slow_melody.wav


### Generate music for multiple prompts:

In [9]:
def generate_music_batch(
    model,
    processor,
    prompt_list,
    output_dir="generated_music",
    guidance_scale=3,
    max_new_tokens=1024,
    device="cuda",
):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    output_paths = []

    # Generate audio for each prompt
    for i, prompt in enumerate(prompt_list):
        prompt_name = prompt["name"]
        prompt_text = prompt["text"]

        print(f"\n[{i + 1}/{len(prompt_list)}] Generating: {prompt_name}")
        print(f"Prompt: {prompt_text}")

        # Process the text prompt
        inputs = processor(
            text=[prompt_text],
            padding=True,
            return_tensors="pt",
        ).to(device)

        # Generate audio
        start_time = time.time()
        audio_values = model.generate(
            **inputs,
            do_sample=True,
            guidance_scale=guidance_scale,
            max_new_tokens=max_new_tokens,
        )
        generation_time = time.time() - start_time

        # Convert to numpy array and cast to float32
        audio_np = audio_values.cpu().numpy().squeeze().astype(np.float32)

        # Save as WAV file
        output_path = os.path.join(output_dir, f"{prompt_name}.wav")
        scipy.io.wavfile.write(output_path, rate=32000, data=audio_np)
        output_paths.append(output_path)

        # Play the audio (in notebook)
        display(Audio(audio_np, rate=32000))

        print(f"✓ Saved to {output_path}")
        print(f"  Generation time: {generation_time:.2f} seconds")

        # Clear GPU memory between generations
        torch.cuda.empty_cache()

        # Small delay between generations
        time.sleep(1)

    print(f"\n✓ Generation complete! All files saved to: {output_dir}")
    return output_paths


In [ ]:
# Define your prompt list
prompts = [
    {
        "name": "01_sitar_led_fusion",
        "text": "A contemplative sitar composition at 75 BPM with melodic phrases based on Raga Manj Khamaj. Gentle tabla rhythms provide a subtle pulse while tanpura creates a warm drone. Incorporate occasional piano accents and minimal electronic textures that enhance but never overwhelm the organic instruments.",
    },
    {
        "name": "02_emotional_centerpiece",
        "text": "A deeply emotional piece featuring intertwining sitar and cello melodies at 65 BPM. Begin with a sparse, meditative introduction before developing into a conversation between Eastern and Western string instruments. Include subtle electronic production elements and ambient textures that create space around the acoustic instruments.",
    },
    {
        "name": "03_rhythmic_expression",
        "text": "Dynamic Indian fusion starting with an improvised sitar alap that gradually introduces syncopated tabla patterns at 80 BPM. Layer in string arrangements that complement the sitar's melodic lines while maintaining traditional Hindustani ornamentation techniques. The piece should build intensity through rhythmic development rather than volume.",
    },
    {
        "name": "04_family_connection",
        "text": "A tender, intimate composition with sitar as the primary voice, supported by gentle piano phrases and subtle string arrangements. Tempo around 70 BPM with a sense of emotional narrative that suggests themes of family and connection. Include moments of space and reflection between melodic statements.",
    },
    {
        "name": "05_cross_cultural_dialogue",
        "text": "A conversation between sitar and acoustic guitar at 72 BPM, supported by tabla and tanpura drone. The composition should move between Indian classical phrases and Western folk-influenced sections, with strings building emotional bridges between these worlds. Include minimalist electronic production elements that add atmospheric texture.",
    },
    {
        "name": "06_meditative_journey",
        "text": "A reflective piece beginning with solo sitar and tanpura at 60 BPM, gradually introducing subtle percussion and ambient electronic textures. The melody should follow the contours of Raga Yaman with its contemplative quality. Create a sense of spaciousness with reverb and delay effects that suggest physical and emotional distance.",
    },
    {
        "name": "07_indian_classical_roots",
        "text": "A composition honoring traditional Hindustani classical music with contemporary production. Sitar explores melodic improvisations over a 16-beat rhythmic cycle (teental) at 85 BPM with supportive tabla accompaniment. Gradually introduce subtle string harmonies and minimal piano accents that complement the traditional framework without overshadowing it.",
    },
    {
        "name": "08_grief_and_memory",
        "text": "A poignant composition at 68 BPM featuring melancholic sitar phrases and emotionally resonant string arrangements. Create a sense of remembrance through recurring melodic motifs and thoughtful use of silence. The piece should convey both loss and celebration with moments of both introspection and release.",
    },
    {
        "name": "09_electronic_acoustic_fusion",
        "text": "A balanced blend of traditional Indian instruments and contemporary electronic elements at 78 BPM. Sitar and tabla form the core, while subtle synthesizer pads, electronic percussion, and processed sounds create an immersive sound environment. Maintain the sitar's lyrical quality as it weaves through both acoustic and electronic landscapes.",
    },
    {
        "name": "10_dawn_raga",
        "text": "A morning raga-inspired piece beginning with the serenity of solo sitar and tanpura at 65 BPM. As the composition unfolds, introduce gentle tabla rhythms, harp-like piano arpeggios, and warm string harmonies that suggest the gradual brightening of day. Create a sense of hope and renewal through ascending melodic patterns and growing instrumental texture.",
    },
]

# Load a model
model, processor = load_fine_tuned_musicgen("as-traes-updated")

# Generate the batch of music
output_paths = generate_music_batch(
    model,
    processor,
    prompts,
    output_dir="indian_fusion_as_traes_updated_set1",
)

In [19]:
def generate_dual_samples_batch(
    prompt_list,
    model,
    processor,
    model_prefix="as-traes",
    output_dir="output/comparison",
    rate=32000,
    guidance_scale=3,
    max_new_tokens=1536,
    device="cuda",
):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Common timestamp for the batch
    timestamp = time.strftime("%Y%m%d-%H%M%S")

    # Normalize prompt_list to handle both string prompts and dict items
    normalized_prompts = []
    for i, item in enumerate(prompt_list):
        if isinstance(item, dict) and "text" in item:
            name = item.get("name", f"prompt_{i + 1}")
            normalized_prompts.append({"name": name, "text": item["text"]})
        else:
            # If just a string, create a simple name
            normalized_prompts.append({"name": f"prompt_{i + 1}", "text": item})

    # Track all output paths
    all_output_paths = []

    # Process each prompt in the list
    for prompt_idx, prompt_item in enumerate(normalized_prompts):
        user_prompt = prompt_item["text"]
        prompt_name = prompt_item["name"]

        print(
            f"\n[{prompt_idx + 1}/{len(normalized_prompts)}] Processing: {prompt_name}"
        )

        # Define the two prompt versions
        base_prompt = user_prompt
        enhanced_prompt = f"Indian classical fusion music in the style of Anoushka Shankar, {user_prompt}"

        # Process each prompt variant (original and enhanced)
        for i, prompt in enumerate([base_prompt, enhanced_prompt]):
            variant = "original" if i == 0 else "enhanced"
            print(f"\nGenerating {variant} version:")
            print(f"Prompt: {prompt}")

            # Process input
            inputs = processor(text=[prompt], padding=True, return_tensors="pt").to(
                device
            )

            # Generate audio
            start_time = time.time()
            audio_values = model.generate(
                **inputs,
                do_sample=True,
                guidance_scale=guidance_scale,
                max_new_tokens=max_new_tokens,
            )
            generation_time = time.time() - start_time

            # Convert and preview
            audio_np = audio_values.cpu().numpy().squeeze().astype(np.float32)
            display(Audio(audio_np, rate=rate))

            # Create sanitized filename
            # Use the prompt_name if available, otherwise sanitize the prompt text
            if prompt_name and prompt_name != f"prompt_{prompt_idx + 1}":
                file_prefix = re.sub(r"[^\w\-_]", "_", prompt_name)
            else:
                file_prefix = re.sub(r"[^\w\-_]", "_", user_prompt[:30])

            output_path = os.path.join(
                output_dir, f"{model_prefix}_{file_prefix}_{variant}_{timestamp}.wav"
            )

            # Save audio file
            scipy.io.wavfile.write(output_path, rate, audio_np)
            all_output_paths.append(output_path)

            print(f"✓ Saved to: {output_path}")
            print(f"  Generation time: {generation_time:.2f} seconds")

            # Clear GPU memory between generations
            torch.cuda.empty_cache()
            time.sleep(0.5)  # Short pause between generations

        print(f"✓ Completed both versions for: {prompt_name}")

    print(f"\n✓ All samples generated successfully!")
    print(f"  Total files generated: {len(all_output_paths)}")
    return all_output_paths


# Example usage:


In [ ]:
model, processor = load_fine_tuned_musicgen("as-traes-updated")

In [ ]:
prompts = [
    {
        "name": "01_sitar_led_fusion",
        "text": "A contemplative sitar composition at 75 BPM with melodic phrases based on Raga Manj Khamaj. Gentle tabla rhythms provide a subtle pulse while tanpura creates a warm drone. Incorporate occasional piano accents and minimal electronic textures that enhance but never overwhelm the organic instruments.",
    },
    {
        "name": "02_emotional_centerpiece",
        "text": "A deeply emotional piece featuring intertwining sitar and cello melodies at 65 BPM. Begin with a sparse, meditative introduction before developing into a conversation between Eastern and Western string instruments. Include subtle electronic production elements and ambient textures that create space around the acoustic instruments.",
    },
    {
        "name": "03_rhythmic_expression",
        "text": "Dynamic Indian fusion starting with an improvised sitar alap that gradually introduces syncopated tabla patterns at 80 BPM. Layer in string arrangements that complement the sitar's melodic lines while maintaining traditional Hindustani ornamentation techniques. The piece should build intensity through rhythmic development rather than volume.",
    },
    {
        "name": "04_family_connection",
        "text": "A tender, intimate composition with sitar as the primary voice, supported by gentle piano phrases and subtle string arrangements. Tempo around 70 BPM with a sense of emotional narrative that suggests themes of family and connection. Include moments of space and reflection between melodic statements.",
    },
    {
        "name": "05_cross_cultural_dialogue",
        "text": "A conversation between sitar and acoustic guitar at 72 BPM, supported by tabla and tanpura drone. The composition should move between Indian classical phrases and Western folk-influenced sections, with strings building emotional bridges between these worlds. Include minimalist electronic production elements that add atmospheric texture.",
    },
    {
        "name": "06_meditative_journey",
        "text": "A reflective piece beginning with solo sitar and tanpura at 60 BPM, gradually introducing subtle percussion and ambient electronic textures. The melody should follow the contours of Raga Yaman with its contemplative quality. Create a sense of spaciousness with reverb and delay effects that suggest physical and emotional distance.",
    },
    {
        "name": "07_indian_classical_roots",
        "text": "A composition honoring traditional Hindustani classical music with contemporary production. Sitar explores melodic improvisations over a 16-beat rhythmic cycle (teental) at 85 BPM with supportive tabla accompaniment. Gradually introduce subtle string harmonies and minimal piano accents that complement the traditional framework without overshadowing it.",
    },
    {
        "name": "08_grief_and_memory",
        "text": "A poignant composition at 68 BPM featuring melancholic sitar phrases and emotionally resonant string arrangements. Create a sense of remembrance through recurring melodic motifs and thoughtful use of silence. The piece should convey both loss and celebration with moments of both introspection and release.",
    },
    {
        "name": "09_electronic_acoustic_fusion",
        "text": "A balanced blend of traditional Indian instruments and contemporary electronic elements at 78 BPM. Sitar and tabla form the core, while subtle synthesizer pads, electronic percussion, and processed sounds create an immersive sound environment. Maintain the sitar's lyrical quality as it weaves through both acoustic and electronic landscapes.",
    },
    {
        "name": "10_dawn_raga",
        "text": "A morning raga-inspired piece beginning with the serenity of solo sitar and tanpura at 65 BPM. As the composition unfolds, introduce gentle tabla rhythms, harp-like piano arpeggios, and warm string harmonies that suggest the gradual brightening of day. Create a sense of hope and renewal through ascending melodic patterns and growing instrumental texture.",
    },
]

# Generate all versions
output_paths = generate_dual_samples_batch(
    prompts,
    model=model,
    processor=processor,
    model_prefix="indian-classical-fusion-as-updated",
    output_dir="output/comparison_batch",
)